In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix,accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, f_classif,chi2
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

df = pd.read_csv('./train1.csv')
X_train = df.drop(['target','id'], axis=1)
y_train = df['target']


In [3]:
df_test = pd.read_csv('test.csv')
X_test = df_test.drop('id',axis=1) 
test_ids = df_test['id']

In [5]:
categorical_cols = [col for col in X_train.columns if col.endswith('_cat')]
X_train[categorical_cols] = X_train[categorical_cols].astype('category')
X_test[categorical_cols] = X_test[categorical_cols].astype('category')
binary_cols = [col for col in X_train.columns if col.endswith('_bin')]
X_train[binary_cols] = X_train[binary_cols].astype(bool)
X_test[binary_cols] = X_test[binary_cols].astype(bool)

In [6]:
def simple_imputer(X_train, X_val, categorical_columns=None, numerical_columns=None):
    """
    Impute missing values in train and validation/test sets using only training data statistics.
    - categorical_columns: list of categorical columns to impute with mode
    - numerical_columns: list of numerical columns to impute with mean
    """
    X_train_imputed = X_train.copy()
    X_val_imputed = X_val.copy()

    # Categorical columns
    if categorical_columns:
        for col in categorical_columns:
            if col in X_train_imputed.columns:
                mode_value = X_train_imputed[col].mode(dropna=True)
                if not mode_value.empty:
                    mode_value = mode_value[0]
                    X_train_imputed[col].fillna(mode_value, inplace=True)
                    if col in X_val_imputed.columns:
                        X_val_imputed[col].fillna(mode_value, inplace=True)

    # Numerical columns
    if numerical_columns:
        for col in numerical_columns:
            if col in X_train_imputed.columns:
                mean_value = X_train_imputed[col].mean(skipna=True)
                if pd.notna(mean_value):  # ensure mean is valid
                    X_train_imputed[col].fillna(mean_value, inplace=True)
                    if col in X_val_imputed.columns:
                        X_val_imputed[col].fillna(mean_value, inplace=True)

    return X_train_imputed, X_val_imputed


In [7]:
num_cols = [col for col in X_train.columns if not col.endswith(('_cat', '_bin'))]

In [8]:
X_train_imputed,X_test_imputed = simple_imputer(X_train, X_test, categorical_cols,num_cols)

In [12]:
selected_features_anova_chi = ['ps_car_13', 'ps_reg_02', 'ps_car_12', 'feature4', 'ps_reg_03', 'feature2', 'ps_car_15', 'ps_ind_15', 'ps_reg_01', 'ps_ind_01', 'feature5', 'ps_car_14', 'feature7', 'ps_ind_03', 'ps_calc_01', 'ps_car_04_cat', 'ps_ind_05_cat', 'ps_car_11_cat', 'ps_car_06_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_ind_04_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_05_cat', 'ps_ind_17_bin', 'ps_ind_07_bin', 'ps_ind_06_bin', 'ps_ind_16_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_12_bin']


In [13]:
tuned_adaboost_model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=2, min_samples_leaf=1, min_samples_split=2),
    learning_rate=0.5,
    n_estimators=200
)
tuned_adaboost_model.fit(X_train_imputed[selected_features_anova_chi], y_train)
y_pred_proba_chi_anova = tuned_adaboost_model.predict_proba(X_test_imputed[selected_features_anova_chi])[:, 1]

submission = pd.DataFrame({'id': test_ids, 'target': y_pred_proba_chi_anova})
submission.to_csv('submission_chi_anova_1.csv', index=False) 

In [15]:
tuned_adaboost_model_chi_anova = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1, min_samples_leaf=1, min_samples_split=2),
    learning_rate=1,
    n_estimators=100
)
tuned_adaboost_model_chi_anova.fit(X_train_imputed[selected_features_anova_chi], y_train)
y_pred_proba_chi_anova = tuned_adaboost_model_chi_anova.predict_proba(X_test_imputed[selected_features_anova_chi])[:, 1]
submission = pd.DataFrame({'id': test_ids, 'target': y_pred_proba_chi_anova})
submission.to_csv('submission_chi_anova_2.csv', index=False) 